In [2]:
!pip install bs4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=0339d8f70db03c6155a3997d63c57321c80bf092b4582cee028c6b5a630be308
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Parse the html data:

In [5]:
soup = BeautifulSoup(html_data, "html.parser")
#print(soup.prettify())

Extract data from the table into three relevant columns:

In [6]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not Assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = ((str((row.span.text).split('(')[1:]).replace(' /',',')).replace(')',' ')).strip(' ')

#        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
#print(table_contents)

Save the data to a dataframe:

In [7]:
df=pd.DataFrame(table_contents)

In [8]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(14)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,[]
1,M2A,Not assigned,[]
2,M3A,North York,['Parkwoods ']
3,M4A,North York,['Victoria Village ']
4,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']"
5,M6A,North York,"['Lawrence Manor, Lawrence Heights ']"
6,M7A,Queen's Park,['Ontario Provincial Government ']
7,M8A,Not assigned,[]
8,M9A,Etobicoke,['Islington Avenue ']
9,M1B,Scarborough,"['Malvern, Rouge ']"


Remove rows where borough not assigned:

In [12]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace=True)

df = df.reset_index(drop=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,['Parkwoods ']
1,M4A,North York,['Victoria Village ']
2,M5A,Downtown Toronto,"['Regent Park, Harbourfront ']"
3,M6A,North York,"['Lawrence Manor, Lawrence Heights ']"
4,M7A,Queen's Park,['Ontario Provincial Government ']
...,...,...,...
98,M8X,Etobicoke,"['The Kingsway, Montgomery Road, Old Mill Nort..."
99,M4Y,Downtown Toronto,['Church and Wellesley ']
100,M7Y,East Toronto Business,['Enclave of M4L ']
101,M8Y,Etobicoke,"[""Old Mill South, King's Mill Park, Sunnylea, ..."


In [11]:
df.shape

(103, 3)